# Рекомендация тарифов

Многие клиенты телеком-компании Заказчика пользуются архивными тарифами. Заказчик хочет построить систему, способную проанализировать поведение клиентов и предложить им один из новых тарифов: *«Смарт»* или *«Ультра»*.

***Задача:*** построить модель для задачи классификации, которая выберет подходящий тариф. Целевая метрика, *accuracy*, не менее 0.75.

***Входные данные:*** предобработанные данные о поведении клиентов, которые уже перешли на новые тарифы.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-из-файла" data-toc-modified-id="Изучение-данных-из-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span></li><li><span><a href="#Проверка-моделей-на-адекватность" data-toc-modified-id="Проверка-моделей-на-адекватность-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на адекватность</a></span></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Изучение данных из файла

Изучим датасет:

In [1]:
# Обновление библиотеки sklearn до последней версии:
#! pip install -U scikit-learn --upgrade

# Импорт необходимых для работы библиотек:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Чтение файла с данными и сохранение в соответствующей переменной
data = pd.read_excel('/datasets/users_behavior.csv')
display(data)
data.info()

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 22.3 MB 38 kB/s  eta 0:00:01


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


###### Вывод

Всего в таблице 5 столбцов со следующими типами данных: float64, int64.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

 - сalls — количество звонков;
 - minutes — суммарная длительность звонков в минутах;
 - messages — количество sms-сообщений;
 - mb_used — израсходованный интернет-трафик в Мб;
 - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разделение данных на выборки

Для контроля переобучения и правильной оценки готовой модели необходимо разделить данные на обучающую, валидационную и тестовую выборки.

Так как в нашей задаче отсутствует подготовленный заранее тестовый набор данных, разобъем исходные данные в следующем соотношении:

 - Обучающая выборка: 60%;
 - Валидационная выборка: 20%;
 - Тестовая выборка: 20%.

In [2]:
# Выделение признаков
features = data.drop(['is_ultra'], axis = 1)

# Выделение целевого признака
target = data['is_ultra']

# Формирование обучающей, валидационной и тестовой выборок
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.4,
    random_state = 12345)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_test,
    target_test,
    test_size=0.5,
    random_state = 12345)

#Вывод на экран размеров полученных выборок
print('Размер обучающей таблицы признаков:',features_train.shape)
print('Размер обучающей таблицы с целевым признаком:',target_train.shape)
print('Размер валидационной таблицы признаков:',features_valid.shape)
print('Размер валидационной таблицы с целевым признаком:',target_valid.shape)
print('Размер тестовой таблицы признаков:',features_test.shape)
print('Размер тестовой таблицы с целевым признаком:',target_test.shape)

Размер обучающей таблицы признаков: (1928, 4)
Размер обучающей таблицы с целевым признаком: (1928,)
Размер валидационной таблицы признаков: (643, 4)
Размер валидационной таблицы с целевым признаком: (643,)
Размер тестовой таблицы признаков: (643, 4)
Размер тестовой таблицы с целевым признаком: (643,)


###### Вывод

Размеры выборок соответствуют соотношению:
 - Обучающая выборка: 60%;
 - Валидационная выборка: 20%;
 - Тестовая выборка: 20%.


## Исследование моделей

Исследуем поведение следующих моделей для задачи классификации:

 - Дерево решений;
 - Случайный лес;
 - Логистическая регрессия.

Исследуем модель ***Дерево решений***:

In [3]:
tree_best_model = None
tree_best_result = 0
for depth in range(1,11):
    # Установка гиперпараметров модели
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # Обучение модели
    tree_model.fit(features_train,target_train)
    # Оценка качества модели на валидационной выборке
    tree_result = tree_model.score(features_valid, target_valid)
    # Определение наилучшей модели
    if tree_result > tree_best_result:
        tree_best_model = tree_model
        tree_best_result = tree_result
# Вывод информации о наилучшей модели на экран
print('Accuracy наилучшей модели алгоритма "Дерево решений" на валидационной выборке: {:.3f}'.format(tree_best_result))
display('Параметры модели:', tree_best_model.get_params())

Accuracy наилучшей модели алгоритма "Дерево решений" на валидационной выборке: 0.799


'Параметры модели:'

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 12345,
 'splitter': 'best'}

Исследуем модель ***Случайный лес***:

In [4]:
forest_best_model = None
forest_best_result = 0
for est in range(1,16):
    for depth in range(1,11):
        # Установка гиперпараметров модели
        forest_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        # Обучение модели
        forest_model.fit(features_train, target_train)
        # Оценка качества модели на валидационной выборке
        forest_result = forest_model.score(features_valid, target_valid)
        # Определение наилучшей модели
        if forest_result > forest_best_result:
            forest_best_model = forest_model
            forest_best_result = forest_result
# Вывод информации о наилучшей модели на экран
print('Accuracy наилучшей модели алгоритма "Случайный лес" на валидационной выборке: {:.3f}'.format(forest_best_result))
display('Параметры модели:', forest_best_model.get_params())

Accuracy наилучшей модели алгоритма "Случайный лес" на валидационной выборке: 0.815


'Параметры модели:'

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 14,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 12345,
 'verbose': 0,
 'warm_start': False}

Исследуем модель ***Логистическая регрессия***:

In [5]:
regression_model = LogisticRegression(random_state = 12345)
 # Обучение модели
regression_model.fit(features_train,target_train)
 # Определение наилучшей модели
regression_result = regression_model.score(features_valid, target_valid)
# Вывод информации о наилучшей модели на экран
print('Accuracy модели алгоритма "Логистическая регрессия" на валидационной выборке: {:.3f}'.format(regression_result))

Accuracy модели алгоритма "Логистическая регрессия" на валидационной выборке: 0.684


###### Вывод

Оценка моделей по качеству (*Accuracy*):

1. ***Случайный лес***. Алгоритм позволил получить наиболее качественную модель: вместо одного решающего дерева алгоритм обучил несколько независимых друг от друга деревьев и принял решение на основе голосования. Наилучший результат (*Accuracy = 0.815*) был получен при количестве независимых деревьев = 14, глубиной = 9;


2. ***Дерево решений***. Наилучший результат (*Accuracy = 0.799*) был получен при глубине дерева = 7. При меньшем количестве дерево недообучается, при большем - переобучается;


3. ***Логистическая регрессия***. Алгоритм обучил модель с самым низким (*Accuracy = 0.684*) качеством предсказания. Модель недообучена.

Оценка моделей по скорости работы:

1. ***Логистическая регрессия***. Модель работает быстрее остальных: у неё меньше всего параметров;


2. ***Дерево решений***. Скорость решающего дерева ниже, чем у логистисеской регрессии и зависит от глубины дерева;


3. ***Случайный лес***. Данная модель медленнее всех по причине большего количества заданных параметров.

## Проверка моделей на адекватность

В случае равного распределения классов в задаче с тарифами (50% класса "0" и 50% класса "1"), *Accuracy* модели, которая  предсказывает все объекты случайным образом, будет равна 0,5. Таким образом, обученные модели, *Accuracy* которых находится в пределах 0.5 будут считаться неадекватными.

Так как в нашем случае неизвестны доли каждого из классов в обучающей выборке, для проверки рассмотренных нами моделей на адекватность сравним их *Accuracy* с константной моделью, предсказывающей метку доминирующего на трейне класса для любого объекта.

Определим доли каждого класса в столбце с целевым признаком обучающей выборки:

In [6]:
# Расчет долей классов "1" и "0" в обучающей выборке
ones = target_train.sum()/len(target_train)
nulls = 1-ones

# Вывод долей на экран
print('Доля класса "1": {:.3f}'.format(ones))
print('Доля класса "0": {:.3f}'.format(nulls))

Доля класса "1": 0.308
Доля класса "0": 0.692


###### Вывод

Как видно из расчетов, класс "0" доминирует в трейне и константной будет считаться модель, предсказывающая "0" для любого объекта. *Accuracy* такой модели будет равна доле объектов класса "0" трейне, т.е. 0.692.

Таким образом, адекватными считаются обученные модели, *Accuracy* которых > 0.692.

Из рассмотренных нами моделей адекватными будут считаться:

 - ***Дерево решений***, *Accuracy* = 0.799;
 - ***Случайный лес***, *Accuracy* = 0.815.
 
Модель ***Логистическая регрессия***, *Accuracy* = 0.684, будет считаться неадекватной и не будет рассматриваться далее, так как даже констатная модель, предсказывающая всегда "0", обладает большим показателем *Accuracy*.

## Проверка моделей на тестовой выборке

In [7]:
# Создание функции для определения качества моделей на тестовой выборке
def model_testing(model_name):
    result = model_name.score(features_test, target_test)
    print('Accuracy модели "{}" на тестовой выборке: {:.3f}'.format(model_name, result))
    
# Применение функции    
for model in [tree_model, forest_model]:    
    model_testing(model)

Accuracy модели "DecisionTreeClassifier(max_depth=10, random_state=12345)" на тестовой выборке: 0.774
Accuracy модели "RandomForestClassifier(max_depth=10, n_estimators=15, random_state=12345)" на тестовой выборке: 0.787


###### Вывод
На тестовой выборке модели ***Дерево решений*** и ***Случайный лес*** показали более низкие результаты, чем на валидационной выборке: *Accuracy* 0.799 -> 0.744 и 0.815 -> 0.787 соответственно, но лидирующей по качеству также является модель ***Случайный лес***.

## Общий вывод

Если клиенту в большей степени важна точность модели, то наилучший выбор - ***Случайный лес***: *Accuracy* на валидационной и тестовой выборках которой составляет 0.815 и 0.787 соответственно.

Если клиенту в большей степени важна скорость работы модели, то наиболее оптимальной является модель ***Дерево решений***: разница в *Accuracy* с моделью ***Случайный лес*** составляет 0.016 и 0.043 на валидационной и тестовой выборках соответственно.

Модель ***Логистическая регрессия***, *Accuracy* = 0.684, считается неадекватной, так как ее качество ниже, чем у константной модели.